# Collecting Data

This notebook collects data about MLS matches from 2018 to 2023 (excludes 2020) and weather data for each match.

The notebook uses two functions that are located in gather_data.py:

**df_from_fbref**: This function uses the requests library to read in a webpage from FBref.com, then uses BeautifulSoup to parse a table that includes every match from a specified MLS season. The table includes the following information:
1. Date
2. Day of the week
3. Round of the season (regular season or a specific playoff round)
4. Home and away team
5. Final score
6. Stadium
7. Attendance
8. Kick off time (local time)

The function also does some processing of the data to prepare it for exploration and modeling:
1. Removes redundant header rows.
2. Splits the score into home and away goals.
3. Converts local time to a decimal.
4. Creates a new column indicating whether a match is the first home match for the home team that season.

Using supplementary data located in mls_stadiums.xlsx, the function does the following:
5. Each team name is converted to a unique integer.
6. The latitude and longitude of the stadium is added to the data.
7. A new column is created that equals attendance divided by stadium capacity.
8. A new column is created that indicates whether a match is a playoff match.

Using supplementary data located in missing_attendance.xlsx, the function does the following:
9. Adds attendance to matches where the attendance is not listed on FBref.com, but is available at the official MLS website.
10. Corrects a couple incorrect stadiums.

Using supplementary data located in mls_rivals.xlsx, the function does the following:
11. Creates a new column that indicates whether the teams in a match are rivals.
12. Creates a new column that indicates whether the teams in a match are from the same conference.

**add_weather_info**: This function takes the output of df_from_fbref and adds weather information. The weather information is gathered from open-meteo.com which has a convenient API for gathering historical weather information. I use the API to gather the following information:
1. Rainfall
2. Snowfall
3. Temperature
4. Cloud cover
5. Windspeed
6. Windgust speed

The weather measurements have a 1 hour resolution. I choose the hour that precedes the kick-off of each match. The function also records the amount of rain and snow that occurred on the day of each match prior to kick-off.

The data from each season is combined into a single pandas DataFrame that is used in other notebooks.

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
%matplotlib inline
from gather_data import *

In [2]:
# 2023 Season
mls23_df = add_weather_info(df_from_fbref(year='current'))
mls23_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,same_conf,rivals,temperature,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum
0,Regular Season,Sat,2023-02-25,15.5,15,2,0,17,28051,Geodis Park,...,1,0,46.2,3.2,0.00,100.0,10.3,29.9,15.3,0.0
1,Regular Season,Sat,2023-02-25,19.5,4,2,1,9,25513,TQL Stadium,...,0,0,50.4,0.0,0.00,18.0,9.0,25.9,0.6,0.0
2,Regular Season,Sat,2023-02-25,19.5,7,0,1,12,19096,Toyota Stadium,...,1,0,44.7,0.0,0.00,100.0,5.6,17.3,4.3,0.0
3,Regular Season,Sat,2023-02-25,19.5,0,2,1,23,67538,Mercedes-Benz Stadium,...,0,0,59.9,0.0,0.00,90.0,8.5,16.9,19.0,0.0
4,Regular Season,Sat,2023-02-25,19.5,20,4,1,6,18510,Subaru Park,...,1,0,32.2,0.0,0.14,95.0,8.2,17.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Round One,Sat,2023-11-11,17.0,9,1,1,22,16374,Shell Energy Stadium,...,1,0,60.1,0.0,0.00,100.0,13.1,23.0,0.1,0.0
532,Round One,Fri,2023-11-10,19.0,24,1,0,7,33048,Lumen Field,...,1,0,49.1,0.1,0.00,97.0,18.1,33.8,3.9,0.0
533,Round One,Sun,2023-11-12,19.0,6,4,2,0,20182,Lower.com Field,...,1,0,52.3,0.0,0.00,0.0,5.4,20.2,0.0,0.0
534,Conference Semifinals,Sat,2023-11-25,17.5,19,0,2,6,25527,Exploria Stadium,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 2022 Season
mls22_df = add_weather_info(df_from_fbref(year=2022))
mls22_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,same_conf,rivals,temperature,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum
0,Regular Season,Sat,2022-02-26,12.5,11,3,0,5,22010,Banc of California Stadium,...,1,0,39.9,0.0,0.0,0,7.2,11.5,0.0,0.0
1,Regular Season,Sat,2022-02-26,13.0,20,1,1,12,17487,Subaru Park,...,0,0,29.2,0.0,0.0,30,9.6,19.8,0.0,0.0
2,Regular Season,Sat,2022-02-26,15.0,23,1,3,18,16146,PayPal Park,...,0,0,33.4,0.0,0.0,13,8.2,12.6,0.0,0.0
3,Regular Season,Sat,2022-02-26,15.5,6,4,0,28,18939,Lower.com Field,...,0,0,29.7,0.0,0.0,0,6.2,16.6,0.0,0.0
4,Regular Season,Sat,2022-02-26,16.5,7,1,1,27,12233,Toyota Stadium,...,0,0,34.3,0.1,0.0,100,10.2,20.9,0.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503,Conference Semifinals,Sun,2022-10-23,13.0,14,1,3,17,19619,Stade Saputo,...,1,0,48.3,0.0,0.0,0,7.9,13.7,0.0,0.0
504,Conference Semifinals,Sun,2022-10-23,19.0,1,2,1,7,20738,Q2 Stadium,...,1,1,83.9,0.0,0.0,95,32.4,58.0,0.2,0.0
505,Conference Finals,Sun,2022-10-30,12.0,11,3,0,1,22175,Banc of California Stadium,...,1,0,54.8,0.0,0.0,2,7.0,9.7,0.0,0.0
506,Conference Finals,Sun,2022-10-30,20.0,20,3,1,17,19770,Subaru Park,...,1,0,61.9,0.0,0.0,14,1.8,9.4,0.0,0.0


In [4]:
# 2021 Season
mls21_df = add_weather_info(df_from_fbref(year=2021))
mls21_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,same_conf,rivals,temperature,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum
0,Regular Season,Sat,2021-04-17,14.0,14,4,2,27,0,DRV PNK Stadium,...,1,1,80.3,0.0,0.0,19,8.5,21.6,0.0,0.00
1,Regular Season,Sat,2021-04-17,15.0,11,2,0,1,4900,Banc of California Stadium,...,1,0,56.7,0.0,0.0,90,6.2,13.7,0.0,0.00
2,Regular Season,Sat,2021-04-17,15.0,19,0,0,0,11503,Exploria Stadium,...,1,0,75.0,0.2,0.0,100,10.7,27.4,0.3,0.00
3,Regular Season,Fri,2021-04-16,18.5,24,4,0,12,7042,Lumen Field,...,1,0,60.2,0.0,0.0,0,9.7,23.4,0.0,0.00
4,Regular Season,Fri,2021-04-16,19.0,9,2,1,23,6376,BBVA Stadium,...,1,0,72.6,1.5,0.0,100,15.8,34.9,3.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,Conference Semifinals,Sun,2021-11-28,17.5,20,1,1,15,19076,Subaru Park,...,1,0,48.4,0.0,0.0,17,7.1,18.4,0.0,1.75
486,Conference Semifinals,Tue,2021-11-30,19.5,16,2,2,17,25509,Gillette Stadium,...,1,0,39.8,0.0,0.0,100,17.2,40.0,0.0,0.00
487,Conference Finals,Sat,2021-12-04,15.5,21,2,0,22,25218,Providence Park,...,1,0,39.5,0.0,0.0,100,10.0,17.6,0.0,0.00
488,Conference Finals,Sun,2021-12-05,15.0,20,1,2,17,19487,Subaru Park,...,1,0,44.9,0.0,0.0,10,6.1,14.8,0.0,0.00


In [5]:
# 2019 Season
mls19_df = add_weather_info(df_from_fbref(year=2019))
mls19_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,same_conf,rivals,temperature,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum
0,Regular Season,Sat,2019-03-02,13.0,20,1,3,27,16498,Talen Energy Stadium,...,1,0,33.3,0.0,0.14,100,11.6,22.7,9.8,1.68
1,Regular Season,Sat,2019-03-02,14.5,19,2,2,17,25527,Orlando City Stadium,...,1,0,67.3,0.0,0.00,100,4.7,13.7,0.0,0.00
2,Regular Season,Sat,2019-03-02,15.0,28,2,3,12,27837,BC Place Stadium,...,1,0,31.4,0.0,0.00,1,7.0,20.2,0.0,0.00
3,Regular Season,Sat,2019-03-02,15.5,7,1,1,16,14327,Toyota Stadium,...,0,0,38.8,0.2,0.00,100,7.9,16.6,0.1,0.00
4,Regular Season,Sat,2019-03-02,16.0,5,3,3,21,12717,Dick's Sporting Goods Park,...,1,0,26.4,0.0,0.07,100,15.7,31.7,0.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,Conference Semifinals,Thu,2019-10-24,19.5,11,5,3,10,22902,Banc of California Stadium,...,1,1,92.1,0.0,0.00,4,9.0,20.2,0.0,0.00
433,Conference Semifinals,Thu,2019-10-24,20.0,0,2,0,20,41507,Mercedes-Benz Stadium,...,1,0,67.1,0.0,0.00,50,12.0,26.3,0.0,0.00
434,Conference Finals,Tue,2019-10-29,19.0,11,1,3,24,22099,Banc of California Stadium,...,1,0,69.9,0.0,0.00,0,11.5,24.8,0.0,0.00
435,Conference Finals,Wed,2019-10-30,20.0,0,1,2,27,44055,Mercedes-Benz Stadium,...,1,0,70.4,0.1,0.00,100,25.4,44.6,8.5,0.00


In [6]:
# 2018 Season
mls18_df = add_weather_info(df_from_fbref(year=2018))
mls18_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,same_conf,rivals,temperature,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum
0,Regular Season,Sat,2018-03-03,13.0,27,0,2,6,26633,BMO Field,...,1,1,27.9,0.0,0.0,2,20.4,36.4,0.0,0.0
1,Regular Season,Sat,2018-03-03,14.5,9,4,0,0,20377,BBVA Compass Stadium,...,0,0,57.4,0.0,0.0,89,14.8,24.1,0.0,0.0
2,Regular Season,Sun,2018-03-04,14.0,24,0,1,11,40070,CenturyLink Field,...,1,0,33.9,0.0,0.0,35,6.5,11.2,0.0,0.0
3,Regular Season,Sun,2018-03-04,15.0,28,2,1,14,27837,BC Place,...,0,0,35.5,0.0,0.0,100,2.9,9.7,0.0,0.0
4,Regular Season,Sat,2018-03-03,19.0,20,2,0,16,16452,Talen Energy Stadium,...,1,0,46.7,0.0,0.0,35,27.2,54.7,0.0,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Conference Finals,Sun,2018-11-25,17.0,0,3,0,18,70016,Mercedes-Benz Stadium,...,1,0,49.5,0.0,0.0,100,6.2,18.0,0.0,0.0
420,Conference Finals,Sun,2018-11-25,16.5,21,0,0,25,21144,Providence Park,...,1,0,37.2,0.0,0.0,21,7.1,12.6,0.0,0.0
421,Conference Finals,Thu,2018-11-29,19.5,18,1,0,0,22137,Red Bull Arena,...,1,0,43.6,0.0,0.0,83,22.4,46.8,0.0,0.0
422,Conference Finals,Thu,2018-11-29,20.5,25,2,3,21,20091,Children's Mercy Park,...,1,0,40.7,0.0,0.0,0,6.0,16.6,0.3,0.0


In [7]:
# Combine each season into a single DataFrame
mlsall_df = pd.concat([mls18_df, mls19_df, mls21_df, mls22_df, mls23_df], axis=0)

# Create columns for year, month, and day
mlsall_df['date_year'] = mlsall_df['date'].apply(lambda x: int(x[0:4]))
mlsall_df['date_month'] = mlsall_df['date'].apply(lambda x: int(x[5:7]))
mlsall_df['date_day'] = mlsall_df['date'].apply(lambda x: int(x[8:10]))
mlsall_df.reset_index(drop=True, inplace=True)
mlsall_df

,round,day,date,local_time,home_team,home_score,away_score,away_team,attendance,stadium,...,rain,snow,cloudcover,windspeed,windgust,rain_sum,snow_sum,date_year,date_month,date_day
0,Regular Season,Sat,2018-03-03,13.0,27,0,2,6,26633,BMO Field,...,0.0,0.0,2.0,20.4,36.4,0.0,0.0,2018,3,3
1,Regular Season,Sat,2018-03-03,14.5,9,4,0,0,20377,BBVA Compass Stadium,...,0.0,0.0,89.0,14.8,24.1,0.0,0.0,2018,3,3
2,Regular Season,Sun,2018-03-04,14.0,24,0,1,11,40070,CenturyLink Field,...,0.0,0.0,35.0,6.5,11.2,0.0,0.0,2018,3,4
3,Regular Season,Sun,2018-03-04,15.0,28,2,1,14,27837,BC Place,...,0.0,0.0,100.0,2.9,9.7,0.0,0.0,2018,3,4
4,Regular Season,Sat,2018-03-03,19.0,20,2,0,16,16452,Talen Energy Stadium,...,0.0,0.0,35.0,27.2,54.7,0.0,1.4,2018,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,Round One,Sat,2023-11-11,17.0,9,1,1,22,16374,Shell Energy Stadium,...,0.0,0.0,100.0,13.1,23.0,0.1,0.0,2023,11,11
2302,Round One,Fri,2023-11-10,19.0,24,1,0,7,33048,Lumen Field,...,0.1,0.0,97.0,18.1,33.8,3.9,0.0,2023,11,10
2303,Round One,Sun,2023-11-12,19.0,6,4,2,0,20182,Lower.com Field,...,0.0,0.0,0.0,5.4,20.2,0.0,0.0,2023,11,12
2304,Conference Semifinals,Sat,2023-11-25,17.5,19,0,2,6,25527,Exploria Stadium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,11,25


In [8]:
# Save data as CSV
mlsall_df.to_csv('mls_with_weather.csv')